# 1. Setup

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import sys
import os
import io
 


import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

# Datetime handling
import datetime


In [3]:
data = pd.read_pickle('BC5DataNoString.pkl')

In [4]:
data.head()

ProductFamily_ID  ProductCategory_ID  ProductBrand_ID  ProductName_ID  \
0                16                  11              306             649   
1                16                  11              306             649   
2                16                  11              306             649   
3                16                  11              306             649   
4                16                  11              306             649   

   ProductPackSKU_ID  Point-of-Sale_ID       Date  Measures   Value  
0               1970                 1 2017-03-04     False     2.0  
1               1970                 1 2017-03-04      True  1540.0  
2               1970                 1 2016-05-02     False     4.0  
3               1970                 1 2016-05-02      True  3080.0  
4               1970                 1 2016-10-24     False     2.0

In [5]:
data = data[['ProductFamily_ID','ProductCategory_ID','ProductBrand_ID','ProductName_ID']]

In [6]:
print(data['ProductFamily_ID'].nunique())
print(data['ProductCategory_ID'].nunique())
print(data['ProductBrand_ID'].nunique())

21
178
1523


In [9]:
group = data.groupby('ProductCategory_ID')
df2 = group.apply(lambda x: x['ProductName_ID'].unique())
df2

ProductCategory_ID
1      [90, 992, 993, 88, 1584, 1517, 356, 1581, 326,...
2      [993, 862, 37, 38, 41, 2006, 357, 34, 1024, 39...
3      [90, 2733, 1144, 690, 1583, 992, 2018, 691, 88...
4               [993, 2019, 1957, 1608, 382, 2036, 2037]
5                                                 [1407]
                             ...                        
174    [22, 30, 1783, 737, 736, 1069, 308, 1788, 1786...
175    [1121, 1120, 24, 21, 736, 1066, 1069, 309, 305...
176                                           [359, 648]
177                       [1294, 1072, 1295, 1293, 1124]
178    [1422, 1014, 1065, 1752, 1021, 1274, 656, 1747...
Length: 178, dtype: object

A quick test shows there are many-to-many relationships among the groups

# 2. Name - Brand Relationship

In [10]:
test = data[['ProductName_ID','ProductBrand_ID']].drop_duplicates()
test.sort_values('ProductName_ID')

ProductName_ID  ProductBrand_ID
4339298               1             1331
1228176               2             1146
692896                3                1
329928                4                1
329960                5                1
...                 ...              ...
26960              2846             1532
243222             2847             1533
204242             2848             1534
20456              2849             1535
24696              2850             1535

[2820 rows x 2 columns]

In [11]:
dup = test['ProductName_ID'].value_counts(ascending=False)
dup

2047    1
1336    1
1110    1
1108    1
1106    1
       ..
473     1
2520    1
471     1
2518    1
2049    1
Name: ProductName_ID, Length: 2820, dtype: int64

In [12]:
dup = test['ProductBrand_ID'].value_counts(ascending=False)
dup

719     60
478     55
189     47
758     37
1472    33
        ..
421      1
423      1
425      1
433      1
2        1
Name: ProductBrand_ID, Length: 1523, dtype: int64

**A Name belongs to exactly 1 Brand**
- Bose Noise Cancelling Headphones 700 would be made only by Bose.

**Multiple names can belong to 1 brand**
- e.g., Samsung can make more than 1 kind of TVs, hence different TV names.

The above findings are coherent

# 3. Name - Category Relationship

In [13]:
test = data[['ProductName_ID','ProductCategory_ID']].drop_duplicates()

In [14]:
dup = test['ProductName_ID'].value_counts(ascending=False)
dup

890     21
48      19
1255    14
1228    13
1304    13
        ..
377      1
375      1
373      1
2420     1
2049     1
Name: ProductName_ID, Length: 2820, dtype: int64

In [15]:
dup = test['ProductCategory_ID'].value_counts(ascending=False)
dup

178    726
109    340
115    170
127    142
22     133
      ... 
142      1
46       1
87       1
158      1
107      1
Name: ProductCategory_ID, Length: 178, dtype: int64

**A Name can belong to multiple Categories**<br/>
**Multiple Names can belong to 1 Category**<br/>
The above findings are coherent

# 4. Name - Family Relationship

In [16]:
test = data[['ProductName_ID','ProductFamily_ID']].drop_duplicates()

In [17]:
dup = test['ProductName_ID'].value_counts(ascending=False)
dup

488     2
1783    2
2254    2
2686    2
21      2
       ..
2516    1
467     1
2514    1
465     1
2049    1
Name: ProductName_ID, Length: 2820, dtype: int64

In [18]:
dup = test['ProductFamily_ID'].value_counts(ascending=False)
dup

21    1891
12     273
1      226
9      197
15      69
18      52
19      28
16      24
2       18
14      16
4        8
6        7
3        6
10       6
11       3
20       3
8        3
5        2
13       2
17       2
7        1
Name: ProductFamily_ID, dtype: int64

**A Name can belong to at most 2 Families**<br/>
**Multiple Names can belong to 1 Family**<br/>
The above findings are coherent

# 5. Brand - Category Relationship

In [19]:
test = data[['ProductBrand_ID','ProductCategory_ID']].drop_duplicates()

In [20]:
dup = test['ProductBrand_ID'].value_counts(ascending=False)
dup

1472    47
719     45
547     35
133     32
668     30
        ..
413      1
415      1
417      1
419      1
2        1
Name: ProductBrand_ID, Length: 1523, dtype: int64

In [21]:
dup = test['ProductCategory_ID'].value_counts(ascending=False)
dup

178    704
109    133
127     94
22      87
115     82
      ... 
70       1
68       1
64       1
120      1
103      1
Name: ProductCategory_ID, Length: 178, dtype: int64

# 6. Brand - Family Relationship

In [22]:
test = data[['ProductBrand_ID','ProductFamily_ID']].drop_duplicates()

In [23]:
dup = test['ProductBrand_ID'].value_counts(ascending=False)
dup

1473    4
324     3
544     2
230     2
1079    2
       ..
1022    1
1023    1
1024    1
1025    1
1       1
Name: ProductBrand_ID, Length: 1523, dtype: int64

In [24]:
dup = test['ProductFamily_ID'].value_counts(ascending=False)
dup

21    698
12    266
1     214
9     193
15     52
16     22
18     20
2      18
14     12
19     11
6       7
4       7
3       6
10      5
8       3
11      3
13      2
20      2
17      2
5       2
7       1
Name: ProductFamily_ID, dtype: int64

# 7. Category - Family Relationship

In [25]:
test = data[['ProductCategory_ID','ProductFamily_ID']].drop_duplicates()

In [26]:
dup = test['ProductCategory_ID'].value_counts(ascending=False)
dup

178    9
27     4
31     3
22     3
14     2
      ..
105    1
104    1
103    1
102    1
89     1
Name: ProductCategory_ID, Length: 178, dtype: int64

In [27]:
dup = test['ProductFamily_ID'].value_counts(ascending=False)
dup

21    166
15      7
19      4
18      4
16      4
14      4
4       3
20      2
1       2
17      2
12      1
13      1
10      1
9       1
8       1
7       1
6       1
5       1
3       1
2       1
11      1
Name: ProductFamily_ID, dtype: int64